# Basic Tensor Operations

In [2]:
import tensorflow as tf
import torch

from utils import compare_tensors, get_torch_and_tf_tensor

for m in [tf, torch]:
    print(f"{m.__name__}=={m.__version__}")

tensorflow==2.0.0
torch==1.3.1


### Math operations

#### Basic single vec operations
Example for `sum`. The same goes for `max`, `min`, `prod`, `mean`.

In [3]:
tto, ttf = get_torch_and_tf_tensor(10, 8, 5, 2)

# Pytorch
tto = tto.sum(dim=1)

# Tensorflow
ttf = tf.reduce_sum(ttf, axis=1)

compare_tensors(tto, ttf)


Tensors are identical
Torch: (10, 5, 2)
Tf:    (10, 5, 2)


### Matrix operations

#### Element wise multiplication `*`, addition `+`, subtraction `-`, division `/`
Mathematical operation will be executed on the last n-dimensions. Caution: The last n dimensions of the second vector must have the same shape as the last n dimensions of the first vector

In [4]:
tto1, ttf1 = get_torch_and_tf_tensor(10, 9, 4, 8)
tto2, ttf2 = get_torch_and_tf_tensor(4, 8)

# Pytorch
tto = tto1 * tto2

# Tensorflow
ttf = ttf1 * ttf2

compare_tensors(tto, ttf)


Tensors are identical
Torch: (10, 9, 4, 8)
Tf:    (10, 9, 4, 8)


#### Matmul
Matrix multiplication of the form `(x, y) * (y, z) -> (x, z)`

In [5]:
tto1, ttf1 = get_torch_and_tf_tensor(5, 2)
tto2, ttf2 = get_torch_and_tf_tensor(2, 10)

# Pytorch
tto = torch.matmul(tto1, tto2)

# Tensorflow
ttf = tf.matmul(ttf1, ttf2)

compare_tensors(tto, ttf)


Tensors are identical
Torch: (5, 10)
Tf:    (5, 10)


#### Simple Batch Matmul
Matrix multiplication of the form `(b, x, y) * (b, y, z) -> (b, x, z)`

In [6]:
tto1, ttf1 = get_torch_and_tf_tensor(10, 5, 2)
tto2, ttf2 = get_torch_and_tf_tensor(10, 2, 10)

# Pytorch
tto = torch.bmm(tto1, tto2)

# Tensorflow
ttf = tf.matmul(ttf1, ttf2)

compare_tensors(tto, ttf)


Tensors are identical
Torch: (10, 5, 10)
Tf:    (10, 5, 10)


#### Extended Batch Matmul
Matrix multiplication of the form `(*dims, x, y) * (*dims, y, z) -> (*dims, x, z)`

In [7]:
tto1, ttf1 = get_torch_and_tf_tensor(10, 8, 5, 2)
tto2, ttf2 = get_torch_and_tf_tensor(10, 8, 2, 10)

# Pytorch
tto = torch.bmm(tto1.view(-1, *tto1.shape[-2:]), 
                tto2.view(-1, *tto2.shape[-2:]))
tto = tto.view(*tto1.shape[:-2], *tto.shape[-2:])

# Tensorflow
ttf = tf.matmul(ttf1, ttf2)

compare_tensors(tto, ttf)


Tensors are identical
Torch: (10, 8, 5, 10)
Tf:    (10, 8, 5, 10)


### Dealing with Tensor shapes

#### Transposing

In [8]:
tto, ttf = get_torch_and_tf_tensor(10, 5, 2)

# Pytorch
tto = tto.transpose(2, 0)

# Tensorflow
ttf = tf.transpose(ttf, (2, 1, 0))

compare_tensors(tto, ttf)


Tensors are identical
Torch: (2, 5, 10)
Tf:    (2, 5, 10)


#### Unsqueeze & Squeeze

In [9]:
tto, ttf = get_torch_and_tf_tensor(10, 5, 2)

# Unsqueeze
tto = tto.unsqueeze(-1)
ttf = tf.expand_dims(ttf, -1)
compare_tensors(tto, ttf)

# Squeeze
tto = tto.squeeze(-1)
ttf = tf.squeeze(ttf, -1)
compare_tensors(tto, ttf)


Tensors are identical
Torch: (10, 5, 2, 1)
Tf:    (10, 5, 2, 1)

Tensors are identical
Torch: (10, 5, 2)
Tf:    (10, 5, 2)


#### Reshaping (Pytorch View)

In [10]:
tto, ttf = get_torch_and_tf_tensor(10, 5, 2)

# Pytorch
tto = tto.view(-1, 2)
tto = tto.view(5, 10, 2)

# Tensorflow
ttf = tf.reshape(ttf, (-1, 2))
ttf = tf.reshape(ttf, (5, 10, 2))

compare_tensors(tto, ttf)


Tensors are identical
Torch: (5, 10, 2)
Tf:    (5, 10, 2)


#### Expanding and Repeating

If dimension that should be repeated is one

In [19]:
tto, ttf = get_torch_and_tf_tensor(10, 1, 2)

# Pytorch
tto = tto.expand(-1, 5, -1)

# Tensorflow
ttf = tf.tile(ttf, (1, 5, 1))

compare_tensors(tto, ttf)


Tensors are identical
Torch: (10, 5, 2)
Tf:    (10, 5, 2)


If dimenstion that should be repeated is not one

In [17]:
tto, ttf = get_torch_and_tf_tensor(10, 2, 2)

# Pytorch
# Note: This is slower than expand and should only be used
#       if the dimension to be repeated is not one
tto = tto.repeat(1, 5, 1)

# Tensorflow
ttf = tf.tile(ttf, (1, 5, 1))

compare_tensors(tto, ttf)


Tensors are identical
Torch: (10, 10, 2)
Tf:    (10, 10, 2)


### Concatenating and Splitting

#### Concatenating

In [13]:
tto1, ttf1 = get_torch_and_tf_tensor(10, 12, 2)
tto2, ttf2 = get_torch_and_tf_tensor(10, 2, 2)

# Pytorch
tto = torch.cat([tto1, tto2], dim=1)

# Tensorflow
ttf = tf.concat([ttf1, ttf2], axis=1)

compare_tensors(tto, ttf)


Tensors are identical
Torch: (10, 14, 2)
Tf:    (10, 14, 2)


#### Splitting

Splitting in equal chunks

In [14]:
tto, ttf = get_torch_and_tf_tensor(8, 10, 5, 2)

# Pytorch
# Caution: 5 determines the size of each section along the specified dimension
tto1, tto2 = torch.split(tto, 5, dim=1)

# Tensorflow
# Caution: 2 determines the number of sections with equal size along the specified axis
ttf1, ttf2 = tf.split(ttf, 2, axis=1)

compare_tensors(tto1, ttf1)
compare_tensors(tto2, ttf2)


Tensors are identical
Torch: (8, 5, 5, 2)
Tf:    (8, 5, 5, 2)

Tensors are identical
Torch: (8, 5, 5, 2)
Tf:    (8, 5, 5, 2)


Splitting into chunks with a list of defined sizes

In [15]:
tto, ttf = get_torch_and_tf_tensor(8, 10, 5, 2)

# Pytorch
ttos = torch.split(tto, [3, 5, 2], dim=1)

# Tensorflow
ttfs = tf.split(ttf, [3, 5, 2], axis=1)

for tto, ttf in zip(ttos, ttfs):
    compare_tensors(tto, ttf)


Tensors are identical
Torch: (8, 3, 5, 2)
Tf:    (8, 3, 5, 2)

Tensors are identical
Torch: (8, 5, 5, 2)
Tf:    (8, 5, 5, 2)

Tensors are identical
Torch: (8, 2, 5, 2)
Tf:    (8, 2, 5, 2)
